<a href="https://colab.research.google.com/github/lblum95/AML/blob/master/task3/Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import  HistGradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [4]:
x_train = pd.read_csv("data/X_train_Jannik.csv", index_col=0, header=0, low_memory=False)
y_train = pd.read_csv("data/y_train.csv", index_col=0, header=0)
x_test = pd.read_csv("data/X_test_Jannik.csv", index_col=0, header=0, low_memory=False)

normal_features=x_train
imputer=SimpleImputer(strategy='median')

scaler = StandardScaler()
norm=normal_features.drop(['HRV_ULF','HRV_VLF','HRV_LF','HRV_LFHF','HRV_LFn'], axis=1)
norm=norm.replace(np.inf, 10)
norm=norm.replace(-np.inf, -10)
norm['y']=y_train
y_tr=norm['y']
norm=norm.drop(['y'], axis=1)
print(norm.shape)
selected=imputer.fit_transform(norm)

(5117, 93)


In [7]:
def get_model():
  estimators = [('l2', HistGradientBoostingClassifier(l2_regularization=3, max_leaf_nodes=None,max_depth=None, min_samples_leaf=10)),('full_steam', HistGradientBoostingClassifier(max_leaf_nodes=None,max_depth=None, min_samples_leaf=10)),('20', HistGradientBoostingClassifier(max_leaf_nodes=None,max_depth=None, min_samples_leaf=20))]
  clf = VotingClassifier(estimators=estimators,voting='soft',flatten_transform=True,n_jobs=-1)
  return clf
skf1=StratifiedKFold(n_splits=5, shuffle=True, random_state=36)
for train_index,test_index in skf1.split(selected,y_tr):
  X_train = selected[train_index]
  X_test = selected[test_index]
  Y_train =y_tr[train_index]
  Y_test = y_tr[test_index]
  model=get_model()
  model.fit(X_train,Y_train)
  Y_pred=model.predict(X_test)
  print(confusion_matrix(Y_test, Y_pred))
  print('Score')
  print(f1_score(Y_test, Y_pred, average='micro'))
  print()
  print()

[[565   2  35   4]
 [  6  63  17   3]
 [ 83  10 199   3]
 [ 14   1   3  16]]
Score
0.8232421875


[[557   2  45   2]
 [  5  67  17   0]
 [ 87   9 197   2]
 [ 13   0   1  20]]
Score
0.8212890625


[[557   4  43   2]
 [  6  54  27   1]
 [ 93  10 189   3]
 [  8   3   4  19]]
Score
0.8005865102639296


[[575   3  27   1]
 [  6  67  14   1]
 [ 94  11 186   4]
 [  9   1   5  19]]
Score
0.8279569892473119


[[573   3  29   1]
 [  9  67  12   1]
 [ 99  14 173   8]
 [ 10   1   2  21]]
Score
0.8152492668621701




In [ ]:
tescht=x_test.drop(['HRV_ULF','HRV_VLF','HRV_LF','HRV_LFHF','HRV_LFn'], axis=1)
tescht=tescht.replace(np.inf, 10)
tescht=tescht.replace(-np.inf, -10)
#maybe change further
tescht=imputer.transform(tescht)
tescht=scaler.transform(tescht)

model = get_model()
model.fit(selected,y_train)
y_tescht=model.predict(tescht)
df = pd.DataFrame(y_tescht)
df.to_csv('data/fsg.csv', header=['y'], index_label='id')